In [1]:
import os
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import skvideo.io
from tqdm import tqdm
import json

In [2]:
folder = '../tmp/threshold0.23'

In [38]:
filename = 'final_result.pkl'
with open(os.path.join(folder, filename), 'rb') as f:
    data = pkl.load(f)
data.pop('seg-id')
results = data
 # {video-id:[], t-start:[], t-end:[], score:[], label:[]}

Get video with two `AllTime` actions

In [4]:
# load labels
json_path = '../data/swallow/anno/swallow_singlestage.json'

In [5]:
with open(json_path, 'r') as f:
    labels = json.load(f)
# filter with test set and two sallow actions
desired_video_label = {}
for vid, label in labels.items():
    if label['subset'] == 'Test':
        # get anno
        annos = label['annotations']
        alltime = []
        for anno in annos:
            if anno['label'] == 'AllTime':
                alltime.append(anno)
        if len(alltime) < 2:
            continue
        else:
            desired_video_label[vid] = alltime
print('Number of videos in test set with two swallow actions:', len(desired_video_label))

Number of videos in test set with two swallow actions: 10


In [6]:
desired_video_label

{'2_16.0_2021062301_kang1nan2e2_jian4kang1cha2ti3_2021_06_23_121717_64': [{'label': 'AllTime',
   'segment': [11.928509446745693, 14.864634843764922],
   'segment(frames)': [357, 445],
   'label_id': 0},
  {'label': 'AllTime',
   'segment': [36.82057254524897, 40.55873219581364],
   'segment(frames)': [1104, 1216],
   'label_id': 0}],
 '5_64.0_2021062305_du4jing4xian2_jian4kang1cha2ti3_2021_06_23_113601_64': [{'label': 'AllTime',
   'segment': [18.688772042985327, 21.397536632624952],
   'segment(frames)': [560, 641],
   'label_id': 0},
  {'label': 'AllTime',
   'segment': [54.43384201999448, 59.68401504445693],
   'segment(frames)': [1631, 1789],
   'label_id': 0}],
 '1_0.0_2021062301_kang1nan2e2_jian4kang1cha2ti3_2021_06_23_121717_64': [{'label': 'AllTime',
   'segment': [27.914638774433683, 30.849234142585264],
   'segment(frames)': [837, 925],
   'label_id': 0},
  {'label': 'AllTime',
   'segment': [52.79373050081684, 56.52994217866105],
   'segment(frames)': [1582, 1694],
   'labe

In [42]:
mask = np.zeros(len(results['video-id']), dtype=bool)
for i, vid in enumerate(results['video-id']):
    if vid in desired_video_label:
        mask[i] = True
# confidence threshold
mask = mask & (results['score'] > 0.3)

# filter results
new_ids = []
for key in results.keys():
    if key == 'video-id':
        for idx in range(len(results[key])):
            if mask[idx]:
                new_ids.append(results[key][idx])
        results[key] = new_ids
    else:
        results[key] = results[key][mask]

In [20]:
len(results)

5

In [43]:
len(results['video-id'])

174

In [44]:
# dump results
# video-id t-start t-end label-id score
with open('../tmp/twoaction-results.txt', 'w') as f:
    for i in range(len(results['video-id'])):
        vid = results['video-id'][i]
        t_start = results['t-start'][i]
        t_end = results['t-end'][i]
        label = results['label'][i]
        score = results['score'][i]
        f.write(f'{vid} {t_start} {t_end} {label} {score}\n')